###Open file

In [16]:
import glob
import os
import csv
import zipfile
import StringIO


s_name = "/Users/ucaiado/Dropbox/NEUTRINO/ALGO/DATA/pisa2012.csv.zip"
s_dict = "/Users/ucaiado/Dropbox/NEUTRINO/ALGO/DATA/pisadict2012.csv"

# for name in glob.glob('C:/Projects/abase/*.zip'):
#     base = os.path.basename(name)
#     filename = os.path.splitext(base)[0]


#     datadirectory = 'C:/Projects/abase/'
#     dataFile = filename
#     archive = '.'.join([dataFile, 'zip'])
#     fullpath = ''.join([datadirectory, archive])
#     csv_file = '.'.join([dataFile, 'csv']) #all fixed


#     filehandle = open(fullpath, 'rb')
#     zfile = zipfile.ZipFile(filehandle)
#     data = StringIO.StringIO(zfile.read(csv_file)) #don't forget this line!
#     reader = csv.reader(data)

#     for row in reader:
#         print row

###Create a function to read the file

In [17]:
import pandas as pd

In [20]:
df_labels = pd.read_csv(s_dict)
df_labels.columns=["VAR","DESCRIPTION"]

In [24]:
df_labels.head(5)

,VAR,DESCRIPTION
0,CNT,Country code 3-character
1,SUBNATIO,Adjudicated sub-region code 7-digit code (3-di...
2,STRATUM,Stratum ID 7-character (cnt + region ID + orig...
3,OECD,OECD country
4,NC,National Centre 6-digit Code


In [36]:
import csv
import zipfile

def _getFields(l_fields, row):
    '''
    Return a dictionary with the data into the row, filtering 
    the fields passed 
    '''
    return {k: row.get(k, None) for k in l_fields}

def getData(s_file, l_fields):
    '''
    Return a list of dictionaries with the data desired
    '''
    with open(s_file, 'rb') as f1:
        zfile = zipfile.ZipFile(f1)
        l_rtn = []
        with zfile.open(zfile.filelist[0].filename) as f:
            reader = csv.DictReader(f)
            for idx, row in enumerate(reader):
                #count lines processed
                if idx%50000==0: print "estou na linha {}".format(idx)
                #get data desired
                l_rtn.append(_getFields(l_fields, row))
        print "estou na linha {}".format(idx)
        
    return l_rtn    

In [37]:
l_test = list(df_labels.head(5).VAR)
%time d_data = getData(s_name, l_test)

estou na linha 0
estou na linha 50000
estou na linha 100000
estou na linha 150000
estou na linha 200000
estou na linha 250000
estou na linha 300000
estou na linha 350000
estou na linha 400000
estou na linha 450000
estou na linha 485489
CPU times: user 2min 18s, sys: 723 ms, total: 2min 19s
Wall time: 2min 19s


###let's see the countries names

In [40]:
df_country =  pd.DataFrame(d_data)

In [44]:
df_country.head()

,CNT,NC,OECD,STRATUM,SUBNATIO
0,Albania,Albania,Non-OECD,ALB0006,0080000
1,Albania,Albania,Non-OECD,ALB0006,0080000
2,Albania,Albania,Non-OECD,ALB0006,0080000
3,Albania,Albania,Non-OECD,ALB0006,0080000
4,Albania,Albania,Non-OECD,ALB0006,0080000


In [76]:
b=[x.strip() for x in df_country.CNT.unique()]
a=[x.strip() for x in df_country.NC.unique()]

In [79]:
[x for  x in a if x not in b]

['United Kingdom (excl.Scotland)',
 'United Kingdom (Scotland)',
 'Republic of Korea',
 'China (Shanghai)',
 'Perm (Russian Federation)',
 'Viet Nam']

In [78]:
[x for  x in b if x not in a]

['United Kingdom',
 'Korea',
 'China-Shanghai',
 'Perm(Russian Federation)',
 'Florida (USA)',
 'Connecticut (USA)',
 'Massachusetts (USA)',
 'Vietnam']

###I like more the CNT names. I will make just some changes

In [82]:
d_newNames = dict(zip(b,b))
d_newNames["United Arab Emirates"] = "UAE"
d_newNames["United Kingdom"] = "UK"
d_newNames["Serbia"] = "Serbia"
d_newNames["Korea"] = "South Korea"
d_newNames["Chinese Taipei"] = "Taiwan"
d_newNames["Slovak Republic"] = "Slovakia"
d_newNames["Russian Federation"] = "Russia"
d_newNames["Perm(Russian Federation)"] = "Russia"
d_newNames["Hong Kong-China"] = "Hong Kong"
d_newNames["China-Shanghai"] = "China"
d_newNames["Macau"] = "Macau"
d_newNames["Connecticut (USA)"] = "USA"
d_newNames["Florida (USA)"] = "USA"
d_newNames["Massachusetts (USA)"] = "USA"
d_newNames["United States of America"] = "USA"

In [86]:
df_country.CNT = df_country.CNT.apply(lambda x: d_newNames[x])

In [89]:
print "Number of Countries in the dataset: {}".format(len(df_country.CNT.unique()))

Number of Countries in the dataset: 64


###Also, I would like to use Gini Index in my visualizations...maybe I could change the color of the countries by it

Here is how it would be done in R
<code>
    > repGINI = vars %>%
    +         group_by(CNT, SUBJECT) %>%
    +         summarise_each(funs(Gini( (./sum(.)) * SCORE)), 
    +                        contains('W_FSTR')) %>%
    +         melt(id=c('CNT', 'SUBJECT'), variable.name='REPNAME', 
    +              value.name='GINI')
</code>

###Look at what there are inside the data

In [57]:
import pprint

In [61]:
pprint.pprint(row.keys())

[None, ',']


In [90]:
with open(s_name, 'rb') as f1:
    zfile = zipfile.ZipFile(f1)
    with zfile.open(zfile.filelist[0].filename) as f:
        reader = csv.DictReader(f)
        for idx, row in enumerate(reader):
            if idx==0:break
    print "estou na linha {}".format(idx)

estou na linha 0


In [12]:
len(row)

2

In [26]:
l_test

['CNT', 'SUBNATIO', 'STRATUM', 'OECD', 'NC']

In [27]:
l_schoolSupport = ["DISCLIMA", "MTSUP", "TEACHSUP", "STUDREL", "TCHBEHFA","TCHBEHSO", "TCHBEHTD"]
l_homeSupport = ["HISCED", "HISEI", "HOMEPOS", "CULTPOS", "ESCS", "HEDRES"]
l_test = list(df_labels.head(5).VAR)

#d_data = {k: row.get(k, None) for k in l_schoolSupport + l_homeSupport + ["PV1MATH"]}
d_data = {k: row.get(k, None) for k in l_test}

In [28]:
d_data.keys()

['OECD', 'CNT', 'STRATUM', 'NC', 'SUBNATIO']

In [29]:
import pprint 
pprint.pprint(d_data)

{'CNT': None, 'NC': None, 'OECD': None, 'STRATUM': None, 'SUBNATIO': None}
